In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import pandas as pd
import os

import _pickle as cPickle
import joblib

In [2]:
data = pd.read_csv('usapl_data.csv')

In [3]:
data

,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,M,SBD,Single-ply,27.0,24-34,24-39,M-O,92.7,93,NaN,...,Yes,USA,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
1,M,SBD,Single-ply,33.5,24-34,24-39,M-O,118.0,120,NaN,...,Yes,USA,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
2,M,SBD,Raw,24.5,24-34,24-39,MR-O,89.6,93,NaN,...,Yes,NaN,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
3,M,SBD,Single-ply,39.5,35-39,40-49,M-M1a,114.8,120,NaN,...,Yes,USA,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
4,M,SBD,Raw,40.5,40-44,40-49,MR-M1a,129.7,120+,NaN,...,Yes,USA,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214194,M,SBD,Single-ply,18.0,18-19,NaN,M-O,82.1,82.5,NaN,...,Yes,USA,MS,USAPL,IPF,2008-08-02,USA,MS,Philadelphia,State Meet
214195,M,SBD,Single-ply,51.0,50-54,50-59,M-O,NaN,125+,NaN,...,Yes,USA,MS,USAPL,IPF,2008-08-02,USA,MS,Philadelphia,State Meet
214196,M,SBD,Single-ply,17.0,16-17,14-18,M-O,82.1,82.5,NaN,...,Yes,USA,MS,USAPL,IPF,2008-08-02,USA,MS,Philadelphia,State Meet
214197,M,SBD,Single-ply,21.0,20-23,19-23,M-O,NaN,125+,NaN,...,Yes,USA,MS,USAPL,IPF,2008-08-02,USA,MS,Philadelphia,State Meet


In [4]:
data.dtypes

Sex                  object
Event                object
Equipment            object
Age                 float64
AgeClass             object
BirthYearClass       object
Division             object
BodyweightKg        float64
WeightClassKg        object
Squat1Kg            float64
Squat2Kg            float64
Squat3Kg            float64
Squat4Kg            float64
Best3SquatKg        float64
Bench1Kg            float64
Bench2Kg            float64
Bench3Kg            float64
Bench4Kg            float64
Best3BenchKg        float64
Deadlift1Kg         float64
Deadlift2Kg         float64
Deadlift3Kg         float64
Deadlift4Kg         float64
Best3DeadliftKg     float64
TotalKg             float64
Place                object
Dots                float64
Wilks               float64
Glossbrenner        float64
Goodlift            float64
Tested               object
Country              object
State                object
Federation           object
ParentFederation     object
Date                

In [5]:
data.isnull().sum()

Sex                      0
Event                    0
Equipment                0
Age                  29496
AgeClass             22925
BirthYearClass       25177
Division                 0
BodyweightKg          2640
WeightClassKg          775
Squat1Kg             69074
Squat2Kg             70052
Squat3Kg             72156
Squat4Kg            214199
Best3SquatKg         34381
Bench1Kg             54061
Bench2Kg             55099
Bench3Kg             57282
Bench4Kg            214156
Best3BenchKg         10547
Deadlift1Kg          65008
Deadlift2Kg          66857
Deadlift3Kg          69539
Deadlift4Kg         214198
Best3DeadliftKg      26065
TotalKg              11179
Place                    0
Dots                 13534
Wilks                13534
Glossbrenner         13534
Goodlift             22445
Tested                   0
Country              36738
State                75454
Federation               0
ParentFederation      4129
Date                     0
MeetCountry              0
M

In [6]:
for column in data.columns:
    if data[column].isnull().sum() > 30000:
#         print(column)
        print(f'{column}: {data[column].isnull().sum()} null values')

Squat1Kg: 69074 null values
Squat2Kg: 70052 null values
Squat3Kg: 72156 null values
Squat4Kg: 214199 null values
Best3SquatKg: 34381 null values
Bench1Kg: 54061 null values
Bench2Kg: 55099 null values
Bench3Kg: 57282 null values
Bench4Kg: 214156 null values
Deadlift1Kg: 65008 null values
Deadlift2Kg: 66857 null values
Deadlift3Kg: 69539 null values
Deadlift4Kg: 214198 null values
Country: 36738 null values
State: 75454 null values
MeetTown: 131581 null values


In [7]:
raw_lifters = data[data['Equipment'] == 'Raw']

In [8]:
raw_lifters

,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
2,M,SBD,Raw,24.5,24-34,24-39,MR-O,89.6,93,NaN,...,Yes,NaN,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
4,M,SBD,Raw,40.5,40-44,40-49,MR-M1a,129.7,120+,NaN,...,Yes,USA,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
5,M,SBD,Raw,46.5,45-49,40-49,MR-M1b,93.4,105,NaN,...,Yes,USA,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
6,F,SBD,Raw,21.5,20-23,19-23,FR-Jr,65.3,72,NaN,...,Yes,NaN,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
8,F,SBD,Raw,40.5,40-44,40-49,FR-M1a,62.7,63,NaN,...,Yes,NaN,NaN,USAPL,IPF,2015-02-07,USA,WA,NaN,Team Phoinix Qualifyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213920,M,SBD,Raw,16.5,16-17,14-18,MR-T2,59.2,60,NaN,...,Yes,USA,NaN,USAPL,IPF,2012-01-01,USA,NaN,NaN,7th Northeastern Regional Powerlifting Champio...
213922,M,SBD,Raw,19.5,20-23,19-23,MR-T3,87.0,90,NaN,...,Yes,NaN,NaN,USAPL,IPF,2012-01-01,USA,NaN,NaN,7th Northeastern Regional Powerlifting Champio...
213926,M,SBD,Raw,12.0,5-12,NaN,MR-Y,51.0,52,NaN,...,Yes,USA,NaN,USAPL,IPF,2012-01-01,USA,NaN,NaN,7th Northeastern Regional Powerlifting Champio...
213933,M,BD,Raw,50.5,50-54,50-59,MR-M2a,99.1,100,NaN,...,Yes,NaN,NaN,USAPL,IPF,2012-01-01,USA,NaN,NaN,7th Northeastern Regional Powerlifting Champio...


In [9]:
raw_lifters.isnull().sum()

Sex                      0
Event                    0
Equipment                0
Age                   4003
AgeClass              3753
BirthYearClass        5901
Division                 0
BodyweightKg           220
WeightClassKg          443
Squat1Kg             23617
Squat2Kg             24263
Squat3Kg             25812
Squat4Kg            140815
Best3SquatKg         16762
Bench1Kg             13805
Bench2Kg             14415
Bench3Kg             16068
Bench4Kg            140815
Best3BenchKg          5113
Deadlift1Kg          19802
Deadlift2Kg          21053
Deadlift3Kg          23027
Deadlift4Kg         140815
Best3DeadliftKg      10904
TotalKg               5812
Place                    0
Dots                  5984
Wilks                 5984
Glossbrenner          5984
Goodlift             11986
Tested                   0
Country               9153
State                22933
Federation               0
ParentFederation      3874
Date                     0
MeetCountry              0
M

In [10]:
# Can drop these columns, too many nan values
for column in raw_lifters.columns:
    if raw_lifters[column].isnull().sum() > 30000:
        print(column)

Squat4Kg
Bench4Kg
Deadlift4Kg
MeetTown


In [11]:
raw_lifters.drop(['Squat4Kg', 'Bench4Kg', 'Deadlift4Kg', 'MeetTown'], axis=1, inplace=True)

/Users/jasonzhang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
clean = raw_lifters.dropna()

In [13]:
# Completely clean dataset
clean.isnull().sum()

Sex                 0
Event               0
Equipment           0
Age                 0
AgeClass            0
BirthYearClass      0
Division            0
BodyweightKg        0
WeightClassKg       0
Squat1Kg            0
Squat2Kg            0
Squat3Kg            0
Best3SquatKg        0
Bench1Kg            0
Bench2Kg            0
Bench3Kg            0
Best3BenchKg        0
Deadlift1Kg         0
Deadlift2Kg         0
Deadlift3Kg         0
Best3DeadliftKg     0
TotalKg             0
Place               0
Dots                0
Wilks               0
Glossbrenner        0
Goodlift            0
Tested              0
Country             0
State               0
Federation          0
ParentFederation    0
Date                0
MeetCountry         0
MeetState           0
MeetName            0
dtype: int64

In [14]:
clean.sort_values(['AgeClass', 'Age'], inplace=True)

<ipython-input-14-430ae4343027>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean.sort_values(['AgeClass', 'Age'], inplace=True)


In [15]:
clean

,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,Goodlift,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetName
1057,M,SBD,Raw,13.0,13-15,14-18,MR-O,63.20,66,102.5,...,56.23,Yes,USA,CT,USAPL,IPF,2017-05-06,USA,CT,Connecticut State Championships and Spring Cla...
1094,M,SBD,Raw,13.0,13-15,14-18,MR-T,63.20,66,102.5,...,56.23,Yes,USA,CT,USAPL,IPF,2017-05-06,USA,CT,Connecticut State Championships and Spring Cla...
4388,F,SBD,Raw,13.0,13-15,14-18,FR-Y3,44.20,47,57.5,...,55.39,Yes,USA,CO,USAPL,IPF,2020-01-19,USA,AZ,Wicked Strong Women
24815,F,SBD,Raw,13.0,13-15,14-18,FR-Y,65.90,72,95.0,...,62.23,Yes,USA,IN,USAPL,IPF,2018-01-27,USA,IN,Hoosier Open
24931,M,SBD,Raw,13.0,13-15,14-18,MR-Y3,94.30,105,-112.5,...,52.95,Yes,USA,CA,USAPL,IPF,2016-03-19,USA,CA,Brand X Method youth/teen/junior Showdown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17187,M,SBD,Raw,85.0,80-999,70-999,MR-M5b,80.60,83,60.0,...,34.07,Yes,USA,MD,USAPL,IPF,2015-10-15,USA,PA,Raw Nationals
96109,M,SBD,Raw,85.5,80-999,70-999,MR-M5,88.70,93,30.0,...,23.10,Yes,USA,NC,USAPL,IPF,2018-08-18,USA,NC,Southeast Regional
126249,F,SBD,Raw,90.0,80-999,70-999,FR-M6,71.34,72,-20.0,...,18.31,Yes,USA,FL,USAPL,IPF,2017-08-05,USA,FL,Southeast Regionals
69019,F,SBD,Raw,91.5,80-999,70-999,FR-M6,70.40,72,-20.0,...,17.42,Yes,USA,FL,USAPL,IPF,2019-02-23,USA,FL,Jacksonville Open Championships


In [ ]:
# clean.to_csv('no_null_vals.csv', index=False)

In [ ]:
clean.dtypes

In [16]:
clean = clean[clean['Sex'] != 'Mx']

clean['AgeClass'].unique()
# 18-19 through 35-39 are the important categories

array(['13-15', '16-17', '18-19', '20-23', '24-34', '35-39', '40-44',
       '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79',
       '80-999'], dtype=object)

In [ ]:
clean.groupby('AgeClass').agg('count')

In [17]:
ages = ['18-19', '20-23', '24-34', '35-39']
clean_ages = clean[clean.AgeClass.isin(ages) == True]

In [19]:
important_cols = ['Sex', 'Age', 'AgeClass', 'BodyweightKg', 'WeightClassKg', 'Best3SquatKg', 'Best3BenchKg', 
                  'Best3DeadliftKg', 'TotalKg', 'Country', 'Date']
clean_ages = clean_ages[important_cols]

In [ ]:
clean_ages['TotalKg'].hist()

In [ ]:
sns.countplot(x='AgeClass', data=clean_ages)
# Most of the data is concentrated between the 18-19 and 35-39 age ranges

In [ ]:
sns.countplot(x='Sex', data=clean_ages)

In [ ]:
clean_ages['Sex'].value_counts()
# Model would not generalize well with the Mx label

In [21]:
clean = clean[important_cols]
clean

,Sex,Age,AgeClass,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,TotalKg,Country,Date
1057,M,13.0,13-15,63.20,66,120.0,77.5,155.0,352.5,USA,2017-05-06
1094,M,13.0,13-15,63.20,66,120.0,77.5,155.0,352.5,USA,2017-05-06
4388,F,13.0,13-15,44.20,47,62.5,42.5,82.5,187.5,USA,2020-01-19
24815,F,13.0,13-15,65.90,72,105.0,60.0,127.5,292.5,USA,2018-01-27
24931,M,13.0,13-15,94.30,105,140.0,110.0,157.5,407.5,USA,2016-03-19
...,...,...,...,...,...,...,...,...,...,...,...
17187,M,85.0,80-999,80.60,83,65.0,80.0,97.5,242.5,USA,2015-10-15
96109,M,85.5,80-999,88.70,93,37.5,55.0,80.0,172.5,USA,2018-08-18
126249,F,90.0,80-999,71.34,72,20.0,30.0,40.0,90.0,USA,2017-08-05
69019,F,91.5,80-999,70.40,72,20.0,30.0,35.0,85.0,USA,2019-02-23


In [20]:
clean_ages

,Sex,Age,AgeClass,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,TotalKg,Country,Date
36,M,17.5,18-19,72.45,74,165.0,107.5,195.0,467.5,USA,2018-11-10
51,M,17.5,18-19,72.45,74,165.0,107.5,195.0,467.5,USA,2018-11-10
328,M,17.5,18-19,68.20,74,160.0,110.0,190.0,460.0,USA,2017-07-22
460,M,17.5,18-19,65.60,67.5,145.0,82.5,170.0,397.5,USA,2016-02-13
462,M,17.5,18-19,74.30,75,107.5,82.5,170.0,360.0,USA,2016-02-13
...,...,...,...,...,...,...,...,...,...,...,...
151383,M,39.5,35-39,104.30,105,197.5,115.0,227.5,540.0,USA,2021-07-24
151384,M,39.5,35-39,104.30,105,197.5,115.0,227.5,540.0,USA,2021-07-24
151396,M,39.5,35-39,104.30,105,197.5,115.0,227.5,540.0,USA,2021-07-24
151984,M,39.5,35-39,83.00,83,187.5,112.5,265.0,565.0,USA,2017-12-02


In [ ]:
clean_ages[clean_ages['Sex'] == 'M']['WeightClassKg'].unique()

In [23]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [29]:
'''
USAPL RECENTLY UPDATED WEIGHT CLASSES, WE CAN DROP THAT BECAUSE BODYWEIGHT ACCOUNTS FOR IT ALREADY
'''



d_features = clean_ages[['Sex', 'Age', 'BodyweightKg', 'Best3SquatKg', 'Best3BenchKg']]
d_target = clean_ages['Best3DeadliftKg']
b_features = clean_ages[['Sex', 'Age', 'BodyweightKg', 'Best3SquatKg', 'Best3DeadliftKg']]
b_target = clean_ages['Best3BenchKg']
s_features = clean_ages[['Sex', 'Age', 'BodyweightKg', 'Best3DeadliftKg', 'Best3BenchKg']]
s_target = clean_ages['Best3SquatKg']
# features = clean[['Sex', 'Age', 'BodyweightKg', 'Best3BenchKg', 'Best3SquatKg']]
# target = clean['Best3DeadliftKg']

# # features_to_encode = ['Sex', 'AgeClass', 'WeightClassKg']
# features_to_encode = ['Sex']
# for feature in features_to_encode:
#     features = encode_and_bind(features, feature)

In [25]:
features

,Age,BodyweightKg,Best3BenchKg,Best3SquatKg,Sex_F,Sex_M
1057,13.0,63.20,77.5,120.0,0,1
1094,13.0,63.20,77.5,120.0,0,1
4388,13.0,44.20,42.5,62.5,1,0
24815,13.0,65.90,60.0,105.0,1,0
24931,13.0,94.30,110.0,140.0,0,1
...,...,...,...,...,...,...
17187,85.0,80.60,80.0,65.0,0,1
96109,85.5,88.70,55.0,37.5,0,1
126249,90.0,71.34,30.0,20.0,1,0
69019,91.5,70.40,30.0,20.0,1,0


In [26]:
# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

# Create the scaler
scaler = MinMaxScaler()

# Fit the scaler to the training data(features only)
scaler.fit(X_train)

# Transform X_train and X_test based on the (same) scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Replace any potential NaN with 0
X_train_scaled[np.isnan(X_train_scaled)] = 0
X_test_scaled[np.isnan(X_test_scaled)] = 0

In [35]:
def train_test_scaled_split(features, target):
    
    features_to_encode = ['Sex']
    for feature in features_to_encode:
        features = encode_and_bind(features, feature)
    
    # split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

    # Create the scaler
    scaler = MinMaxScaler()

    # Fit the scaler to the training data(features only)
    scaler.fit(X_train)

    # Transform X_train and X_test based on the (same) scaler
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Replace any potential NaN with 0
    X_train_scaled[np.isnan(X_train_scaled)] = 0
    X_test_scaled[np.isnan(X_test_scaled)] = 0
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [36]:
pred_map = {'Squat': [s_features, s_target], 'Bench': [b_features, b_target], 'Deadlift': [d_features, d_target]}

In [44]:
for lift, data in pred_map.items():
    
    features, target = data
    
    X_train_scaled, X_test_scaled, y_train, y_test = train_test_scaled_split(features, target)
    
    model = RandomForestRegressor().fit(X=X_train_scaled, y=y_train)

    # Prediction results
    print(lift + ":\n")
    print("\tR-squared value for training set: ", r2_score(y_train, model.predict(X_train_scaled)))
    print("\tMean-squared-error value for training set: ", mean_squared_error(y_train, model.predict(X_train_scaled)))
    print("\n")
    print("\tR-squared value for testing set: ", r2_score(y_test, model.predict(X_test_scaled)))
    print("\tMean-squared-error value for testing set: ", mean_squared_error(y_test, model.predict(X_test_scaled)))
    print("\n")
    best_models[lift] = model

Squat:

	R-squared value for training set:  0.9880029189813558
	Mean-squared-error value for training set:  36.0477692496283


	R-squared value for testing set:  0.9141256802801293
	Mean-squared-error value for testing set:  256.1600783096232


Bench:

	R-squared value for training set:  0.9858974917505587
	Mean-squared-error value for training set:  23.264485775352373


	R-squared value for testing set:  0.9019421399583025
	Mean-squared-error value for testing set:  161.41157918884176


Deadlift:

	R-squared value for training set:  0.9866891980490656
	Mean-squared-error value for training set:  41.68250033911725


	R-squared value for testing set:  0.9029214322276652
	Mean-squared-error value for testing set:  304.05042151882947




In [40]:
best_models = {}

In [41]:
scalers = {}

In [45]:
best_models

{'Squat': RandomForestRegressor(),
 'Bench': RandomForestRegressor(),
 'Deadlift': RandomForestRegressor()}

In [48]:
scalers

{'Squat': MinMaxScaler(), 'Bench': MinMaxScaler(), 'Deadlift': MinMaxScaler()}

In [49]:
# Export models and scalers

for lift, model in best_models.items():
    with open(f'{lift}_model.pickle', 'wb') as output_file:
        cPickle.dump(model, output_file)
    
for lift, scaler in scalers.items():
    scaler_filename = f'{lift}_scaler'
    joblib.dump(scaler, scaler_filename)
    

In [47]:
for lift, data in pred_map.items():
    
    features, target = data
    
    features_to_encode = ['Sex']
    
    for feature in features_to_encode:
        features = encode_and_bind(features, feature)
    
    # split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

    # Create the scaler
    scaler = MinMaxScaler()

    # Fit the scaler to the training data(features only)
    scaler.fit(X_train)
    
    scalers[lift] = scaler
    
    

In [ ]:
scaler_filename = f'..{os.path.sep}models{os.path.sep}tennis_minmax_scaler'
joblib.dump(scaler, scaler_filename)

In [ ]:
with open(f'..{os.path.sep}models{os.path.sep}logreg_model.pickle', 'wb') as output_file:
    cPickle.dump(best_models['Logistic Regression'], output_file)

In [ ]:
best_param_models = {}

In [ ]:
scoring = {"Max Error": "max_error", "R-squared": "r2"}


for lift, data in pred_map.items():
    
    features, target = data
    
    X_train_scaled, X_test_scaled, y_train, y_test = train_test_scaled_split(features, target)
    
    param_grid = {"max_depth":[3, 5, 7, 9, 11]}
    
    grid_search = GridSearchCV(RandomForestRegressor(), param_grid, scoring=scoring, refit='R-squared', return_train_score=True, cv=5)
    
    # Fit the grid search object on the training data (CV will be performed on this)
    grid_search.fit(X=X_train_scaled, y=y_train)
    
    # Grid search results
    print(lift + ":\n")
    print("\tBest estimator: ", grid_search.best_estimator_)
    print("\tBest parameters: ", grid_search.best_params_)
    print("\tBest cross-validation score: ", grid_search.best_score_)
    print("\n")
    
    model = grid_search.best_estimator_
    print("\tR-squared value for training set: ", r2_score(y_train, model.predict(X_train_scaled)))
    print("\tMean-squared-error value for training set: ", mean_squared_error(y_train, model.predict(X_train_scaled)))
    print("\n")
    
    # Add the best model to dictionary
    best_param_models[estimator_name] = grid_search.best_estimator_

In [ ]:
# Parameter grids for Validation/Optimization
ridge_param_grid = {"alpha":[0.001, 0.01, 0.1, 1, 10, 100]}
lasso_param_grid = {"alpha":[0.001, 0.01, 0.1, 1, 10, 100]}
knn_param_grid = {"n_neighbors":[1, 5, 10], "metric": ['euclidean', 'manhattan', 'minkowski']}
tree_param_grid = {"max_depth":[3, 5, 7, 9, 11]}
forest_param_grid = {"max_depth":[3, 5, 7, 9, 11]}

# Dictionary of models with their parameter grids
estimators = {
    'Ridge': [Ridge(), ridge_param_grid],
    'Lasso': [Lasso(), lasso_param_grid],
    'k-Nearest Neighbor': [KNeighborsRegressor(), knn_param_grid],
    'Decision Tree': [DecisionTreeRegressor(), tree_param_grid],
    'Random Forest': [RandomForestRegressor(), forest_param_grid]}

In [ ]:
# Initial Model Performance Analysis

print("Initial Results for Models Trained on All Features\n")
for estimator_name, estimator_objects in estimators.items():
    estimator_model = estimator_objects[0]

    model = estimator_model.fit(X=X_train_scaled, y=y_train)

    # Prediction results
    print(estimator_name + ":\n")
    print("\tR-squared value for training set: ", r2_score(y_train, model.predict(X_train_scaled)))
    print("\tMean-squared-error value for training set: ", mean_squared_error(y_train, model.predict(X_train_scaled)))
    print("\n")
    print("\tR-squared value for testing set: ", r2_score(y_test, model.predict(X_test_scaled)))
    print("\tMean-squared-error value for testing set: ", mean_squared_error(y_test, model.predict(X_test_scaled)))
    print("\n")

In [ ]:
'''
FOR DEADLIFT RANDOM FOREST IS BEST
Initial Results for Models Trained on All Features

Ridge:

	R-squared value for training set:  0.900791795329309
	Mean-squared-error value for training set:  310.66843606211154


	R-squared value for testing set:  0.8982009899083301
	Mean-squared-error value for testing set:  318.8348637482929


Lasso:

	R-squared value for training set:  0.8604311108121666
	Mean-squared-error value for training set:  437.057082837424


	R-squared value for testing set:  0.8591595054946677
	Mean-squared-error value for testing set:  441.11293258562176


k-Nearest Neighbor:

	R-squared value for training set:  0.9268768144787918
	Mean-squared-error value for training set:  228.9837394110679


	R-squared value for testing set:  0.885514033615932
	Mean-squared-error value for testing set:  358.57045623808995
    
    
Decision Tree:

	R-squared value for training set:  0.9997456336932952
	Mean-squared-error value for training set:  0.796542815719662


	R-squared value for testing set:  0.8247594118265117
	Mean-squared-error value for testing set:  548.8541490054909


Random Forest:

	R-squared value for training set:  0.9867339096710506
	Mean-squared-error value for training set:  41.54248682186873


	R-squared value for testing set:  0.9029074378444212
	Mean-squared-error value for testing set:  304.09425197720844
    
    
    
    
    
WITHOUT CLASSES
Ridge:

	R-squared value for training set:  0.898504968343617
	Mean-squared-error value for training set:  317.8295873554723


	R-squared value for testing set:  0.8962298550449012
	Mean-squared-error value for testing set:  325.0084651914204


Lasso:

	R-squared value for training set:  0.8604159502657829
	Mean-squared-error value for training set:  437.10455777410425


	R-squared value for testing set:  0.8591452403723664
	Mean-squared-error value for testing set:  441.1576109996967


k-Nearest Neighbor:

	R-squared value for training set:  0.9281983657588784
	Mean-squared-error value for training set:  224.84532897693873


	R-squared value for testing set:  0.8878280908548475
	Mean-squared-error value for testing set:  351.32282068828425


Decision Tree:

	R-squared value for training set:  0.9997418966085204
	Mean-squared-error value for training set:  0.808245419211549


	R-squared value for testing set:  0.8250143648717643
	Mean-squared-error value for testing set:  548.0556351557776


Random Forest:

	R-squared value for training set:  0.986720766316167
	Mean-squared-error value for training set:  41.58364496518781


	R-squared value for testing set:  0.9025839068349805
	Mean-squared-error value for testing set:  305.1075522560664
'''

In [ ]:
'''
FOR BENCH
Ridge:

	R-squared value for training set:  0.892988866535197
	Mean-squared-error value for training set:  176.53306406644927


	R-squared value for testing set:  0.8964825991149635
	Mean-squared-error value for testing set:  170.39844784776005


Lasso:

	R-squared value for training set:  0.8145834294703707
	Mean-squared-error value for training set:  305.8761669415857


	R-squared value for testing set:  0.8166275930373708
	Mean-squared-error value for testing set:  301.8465809360993


k-Nearest Neighbor:

	R-squared value for training set:  0.9226421141450587
	Mean-squared-error value for training set:  127.61498899707571


	R-squared value for testing set:  0.8842375523053484
	Mean-squared-error value for testing set:  190.55483655480288


Decision Tree:

	R-squared value for training set:  0.9997902915333114
	Mean-squared-error value for training set:  0.3459497809858721


	R-squared value for testing set:  0.8164845794658392
	Mean-squared-error value for testing set:  302.0819934406811


Random Forest:

	R-squared value for training set:  0.9858845429595768
	Mean-squared-error value for training set:  23.285847008281845


	R-squared value for testing set:  0.9021000538156578
	Mean-squared-error value for testing set:  161.1516395462605
    
    
    
    
    
WITHOUT CLASSES
Ridge:

	R-squared value for training set:  0.8904680622199737
	Mean-squared-error value for training set:  180.69155949837258


	R-squared value for testing set:  0.8941704889775871
	Mean-squared-error value for testing set:  174.20437781985777


Lasso:

	R-squared value for training set:  0.8139401495175196
	Mean-squared-error value for training set:  306.9373666266323


	R-squared value for testing set:  0.8160193189895053
	Mean-squared-error value for testing set:  302.8478517633823


k-Nearest Neighbor:

	R-squared value for training set:  0.9230868882485617
	Mean-squared-error value for training set:  126.88125847047938


	R-squared value for testing set:  0.8861075022450645
	Mean-squared-error value for testing set:  187.47673988162134


Decision Tree:

	R-squared value for training set:  0.9997902564728233
	Mean-squared-error value for training set:  0.346007619223888


	R-squared value for testing set:  0.8173917138206432
	Mean-squared-error value for testing set:  300.58877312480723


Random Forest:

	R-squared value for training set:  0.9858982376597523
	Mean-squared-error value for training set:  23.26325527128061


	R-squared value for testing set:  0.9018881160223156
	Mean-squared-error value for testing set:  161.50050718317019
'''

In [ ]:
'''
FOR SQUAT
Ridge:

	R-squared value for training set:  0.9105562760189563
	Mean-squared-error value for training set:  268.75260056053975


	R-squared value for testing set:  0.9104550067401644
	Mean-squared-error value for testing set:  267.1095685007973


Lasso:

	R-squared value for training set:  0.8794849681241365
	Mean-squared-error value for training set:  362.11292175333625


	R-squared value for testing set:  0.8809785596583386
	Mean-squared-error value for testing set:  355.036774415219


k-Nearest Neighbor:

	R-squared value for training set:  0.9340890035860671
	Mean-squared-error value for training set:  198.04353959518784


	R-squared value for testing set:  0.8993721303492088
	Mean-squared-error value for testing set:  300.1693993496946


Decision Tree:

	R-squared value for training set:  0.9998465488672468
	Mean-squared-error value for training set:  0.46107640816830153


	R-squared value for testing set:  0.842712904857357
	Mean-squared-error value for testing set:  469.1818781244988


Random Forest:

	R-squared value for training set:  0.98807146372382
	Mean-squared-error value for training set:  35.84181206256046


	R-squared value for testing set:  0.9142998788291369
	Mean-squared-error value for testing set:  255.6404501588471
    
    
    
    
WITHOUT CLASSES
Ridge:

	R-squared value for training set:  0.9100313257045812
	Mean-squared-error value for training set:  270.3299248922422


	R-squared value for testing set:  0.9101631812638482
	Mean-squared-error value for testing set:  267.980073642611


Lasso:

	R-squared value for training set:  0.879482610475747
	Mean-squared-error value for training set:  362.12000580694723


	R-squared value for testing set:  0.8809764546205674
	Mean-squared-error value for testing set:  355.04305366892487


k-Nearest Neighbor:

	R-squared value for training set:  0.9353216809041093
	Mean-squared-error value for training set:  194.33969968188046


	R-squared value for testing set:  0.9005989534753995
	Mean-squared-error value for testing set:  296.50982907184897


Decision Tree:

	R-squared value for training set:  0.9998460483885462
	Mean-squared-error value for training set:  0.4625802023567154


	R-squared value for testing set:  0.8425767179090207
	Mean-squared-error value for testing set:  469.5881190061077


Random Forest:

	R-squared value for training set:  0.9879984822578319
	Mean-squared-error value for training set:  36.06110032454263


	R-squared value for testing set:  0.9139984869229377
	Mean-squared-error value for testing set:  256.5394916248604
'''

In [27]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Parameter grids for Validation/Optimization
tree_param_grid = {"max_depth":[3, 5, 7, 9, 11]}
forest_param_grid = {"max_depth":[3, 5, 7, 9, 11]}

# Dictionary of models with their parameter grids
estimators = {
    'Decision Tree': [DecisionTreeRegressor(), tree_param_grid],
    'Random Forest': [RandomForestRegressor(), forest_param_grid]}

In [28]:
# Initial Model Performance Analysis

print("Initial Results for Models Trained on All Features\n")
for estimator_name, estimator_objects in estimators.items():
    estimator_model = estimator_objects[0]

    model = estimator_model.fit(X=X_train_scaled, y=y_train)

    # Prediction results
    print(estimator_name + ":\n")
    print("\tR-squared value for training set: ", r2_score(y_train, model.predict(X_train_scaled)))
    print("\tMean-squared-error value for training set: ", mean_squared_error(y_train, model.predict(X_train_scaled)))
    print("\n")
    print("\tR-squared value for testing set: ", r2_score(y_test, model.predict(X_test_scaled)))
    print("\tMean-squared-error value for testing set: ", mean_squared_error(y_test, model.predict(X_test_scaled)))
    print("\n")

Initial Results for Models Trained on All Features

Decision Tree:

	R-squared value for training set:  0.9997907236072651
	Mean-squared-error value for training set:  0.674819641804105


	R-squared value for testing set:  0.8455675156370464
	Mean-squared-error value for testing set:  498.8730074944358


Random Forest:

	R-squared value for training set:  0.9877650709831921
	Mean-squared-error value for training set:  39.45199125769952


	R-squared value for testing set:  0.914729082044128
	Mean-squared-error value for testing set:  275.45603159811503




In [ ]:
# Random Forest does the best out of all 5 models
'''
Initial Results for Models Trained on All Features

Decision Tree:

	R-squared value for training set:  0.9997456336932952
	Mean-squared-error value for training set:  0.796542815719662


	R-squared value for testing set:  0.8247594118265117
	Mean-squared-error value for testing set:  548.8541490054909


Random Forest:

	R-squared value for training set:  0.9867339096710506
	Mean-squared-error value for training set:  41.54248682186873


	R-squared value for testing set:  0.9029074378444212
	Mean-squared-error value for testing set:  304.09425197720844
'''